# Valorant VCT Predictions

## Backstory
The first file I was able to create a model that can predict with up to 70% accuracy. Since then I learned quite a bit about data analytics and figured it would be easier to start fresh when training my next models.

Goal for this file is to get up to 80% accuracy and I will try to explore different models outside of logistic regression.